In [39]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import librosa
import numpy as np    
import os
import signal
import subprocess
import json
import uuid
from IPython.display import HTML

Extractor (Input Validation, Chunking, Mapping From file_csv and annotations_csv to chunks)

In [41]:
from acoustic_knowledge_discovery import extractors

In [42]:
extractor = extractors.Extractor(
    #"sample_inputs/file.csv", #file metadata csv
    "/home/s.kamboj.400/mount/gorongosa_recordings_small_test.csv",
    #"/home/s.kamboj.400/mount/mexico_recordings.csv",
    #"sample_inputs/anno.csv", #annoations csv
    "/home/s.kamboj.400/mount/gorongosa_recordings_anno.csv",
    #"/home/s.kamboj.400/mount/mexico_recordings_anno.csv",
    #"sample_inputs/inputDtory",#input directory with a files subdirectory for audio files & a XC-templates subdirectory for XC templates
    "/home/s.kamboj.400/mount/",
    #"/home/s.kamboj.400/unzipped-coral/PaolaMexico/",
    chunk_size = 5) #chunk size in seconds
chunkDS = extractor.forward()


Casting the dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

In [43]:
# for row in chunkDS.chunk_ds["train"]:
#     print(row)
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', 'Date', 'Time', 'Duration', 'Season', 'Time of Day', 'Audio Hardware', 'Sound Type', 'Location Name', 'Coordinates', 'Audio Channels', 'Sample Rate', 'Bit Depth', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence'],
        num_rows: 8
    })
})

Chunk DS Augmentations
- NOTE: Old because we are now computing on the fly not storing in memory

In [ ]:
# def add_raw_audio_batched(batch):
#     raws = []
#     for path, start in zip(batch["file_path"], batch["chunk_start"]):
#         # os.fspath handles both str and Path
#         #print("os.fspath(path) is ", os.fspath(path))
#         y, _ = librosa.load(
#             os.fspath(path),
#             offset=float(start),
#             duration=float(extractor.chunk_size),
#             sr=None         # keep original sampling rate     
#         )
#         raws.append(y.astype(np.float32))

#     return {"raw_audio": raws}


In [ ]:
# # Add raw audio column in dataset
# chunkDS.chunk_ds = chunkDS.chunk_ds.map(add_raw_audio_batched, 
#                                         batched=True, 
#                                         num_proc=1) #increasing this number a small amount can make it faster
#                                         # ONLY INCREASE ON MAC OR LINUX

In [ ]:
# chunkDS.chunk_ds #make sure that new column raw_audio is added

File-level Operations

In [47]:
from acoustic_knowledge_discovery.features.feature_preprocessing_pipeline import FeatureSequential
from acoustic_knowledge_discovery import features

THRESHOLD = 0.6

pipeline = FeatureSequential(
    features.TemplateMatching(
        #extractor.base_dir / "XC-templates", # link to directory with XC templates
        "/home/s.kamboj.400/mount/XC_results/recordings/",
        THRESHOLD,
        chunk_size=extractor.chunk_size
    ),
    # features.anotherFunctionThatInheretsFromFeaturePreprocessor
)

chunkDS = pipeline.forward(chunkDS)  

366 total templates
Filtered down to 366 unique species templates
  Accipiter tachiro_652454: Accipiter tachiro_652454.mp3
  Acridotheres tristis_947467: Acridotheres tristis_947467.mp3
  Acrocephalus arundinaceus_524528: Acrocephalus arundinaceus_524528.mp3
  Acrocephalus baeticatus_432469: Acrocephalus baeticatus_432469.mp3
  Acrocephalus baeticatus_432471: Acrocephalus baeticatus_432471.mp3
  Acrocephalus baeticatus_432472: Acrocephalus baeticatus_432472.mp3
  Acrocephalus baeticatus_432680: Acrocephalus baeticatus_432680.mp3
  Acrocephalus baeticatus_432681: Acrocephalus baeticatus_432681.mp3
  Acrocephalus baeticatus_432682: Acrocephalus baeticatus_432682.mp3
  Acrocephalus baeticatus_432684: Acrocephalus baeticatus_432684.mp3
  Acrocephalus baeticatus_453372: Acrocephalus baeticatus_453372.mp3
  Acrocephalus gracilirostris_201497: Acrocephalus gracilirostris_201497.mp3
  Acrocephalus gracilirostris_433951: Acrocephalus gracilirostris_433951.mp3
  Acrocephalus gracilirostris_45290

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

...

Progress: 287/366 templates completed (78.4%)
Progress: 288/366 templates completed (78.7%)
Progress: 289/366 templates completed (79.0%)
Progress: 290/366 templates completed (79.2%)
Progress: 291/366 templates completed (79.5%)
Progress: 292/366 templates completed (79.8%)
Progress: 293/366 templates completed (80.1%)
Progress: 294/366 templates completed (80.3%)
Progress: 295/366 templates completed (80.6%)
Progress: 296/366 templates completed (80.9%)
Progress: 297/366 templates completed (81.1%)
Progress: 298/366 templates completed (81.4%)
Progress: 299/366 templates completed (81.7%)
Progress: 300/366 templates completed (82.0%)
Progress: 301/366 templates completed (82.2%)
Progress: 302/366 templates completed (82.5%)
Progress: 303/366 templates completed (82.8%)
Progress: 304/366 templates completed (83.1%)
Progress: 305/366 templates completed (83.3%)
Progress: 306/366 templates completed (83.6%)
Progress: 307/366 templates completed (83.9%)
Progress: 308/366 templates comple

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

Progress: 323/366 templates completed (88.3%)
Progress: 324/366 templates completed (88.5%)
Progress: 325/366 templates completed (88.8%)
Progress: 326/366 templates completed (89.1%)
Progress: 327/366 templates completed (89.3%)
Progress: 328/366 templates completed (89.6%)
Progress: 329/366 templates completed (89.9%)
Progress: 330/366 templates completed (90.2%)
Progress: 331/366 templates completed (90.4%)
Progress: 332/366 templates completed (90.7%)
Progress: 333/366 templates completed (91.0%)
Progress: 334/366 templates completed (91.3%)
Progress: 335/366 templates completed (91.5%)
Progress: 336/366 templates completed (91.8%)
Progress: 337/366 templates completed (92.1%)
Progress: 338/366 templates completed (92.3%)
Progress: 339/366 templates completed (92.6%)
Progress: 340/366 templates completed (92.9%)


[src/libmpg123/parse.c:skip_junk():1260] error: Giving up searching valid MPEG header after 65536 bytes of junk.
/home/s.kamboj.400/acoustic_knowledge_discovery/acoustic_knowledge_discovery/features/template_matching.py:23: UserWarning: PySoundFile failed. Trying audioread instead.
  class TemplateMatching(FeaturePreprocessor):
/home/s.kamboj.400/acoustic_knowledge_discovery/.venv/lib/python3.12/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Progress: 341/366 templates completed (93.2%)
Progress: 342/366 templates completed (93.4%)
Progress: 343/366 templates completed (93.7%)
Progress: 344/366 templates completed (94.0%)
Progress: 345/366 templates completed (94.3%)
Progress: 346/366 templates completed (94.5%)
Progress: 347/366 templates completed (94.8%)
Progress: 348/366 templates completed (95.1%)
Progress: 349/366 templates completed (95.4%)
Progress: 350/366 templates completed (95.6%)
Progress: 351/366 templates completed (95.9%)
Progress: 352/366 templates completed (96.2%)
Progress: 353/366 templates completed (96.4%)
Progress: 354/366 templates completed (96.7%)
Progress: 355/366 templates completed (97.0%)
Progress: 356/366 templates completed (97.3%)
Progress: 357/366 templates completed (97.5%)
Progress: 358/366 templates completed (97.8%)
Progress: 359/366 templates completed (98.1%)
Progress: 360/366 templates completed (98.4%)
Progress: 361/366 templates completed (98.6%)
Progress: 362/366 templates comple

In [48]:
# #verify properly loaded after feature extraction
# for row in chunkDS.chunk_ds["train"]:
#     print(row["Annotation"], row["Confidence"])
#     # print(row)
print(chunkDS.chunk_ds["train"][:5]) # print first 5 rows

{'file_name': ['SERRA_20250503_152505.wav', 'SERRA_20250503_160206.wav', 'SERRA_20250503_160206.wav', 'SERRA_20250503_220842.wav', 'SERRA_20250503_220842.wav'], 'Date': ['2025-05-03', '2025-05-03', '2025-05-03', '2025-05-03', '2025-05-03'], 'Time': ['15:25:05', '16:02:06', '16:02:06', '22:08:42', '22:08:42'], 'Duration': ['00:00:05', '00:00:11', '00:00:11', '00:00:15', '00:00:15'], 'Season': ['Transition Rainy to Dry', 'Transition Rainy to Dry', 'Transition Rainy to Dry', 'Transition Rainy to Dry', 'Transition Rainy to Dry'], 'Time of Day': ['Afternoon', 'Afternoon', 'Afternoon', 'Night', 'Night'], 'Audio Hardware': ['Wildlife Accoustics', 'Wildlife Accoustics', 'Wildlife Accoustics', 'Wildlife Accoustics', 'Wildlife Accoustics'], 'Sound Type': ['Ultrasonic', 'Ultrasonic', 'Ultrasonic', 'Ultrasonic', 'Ultrasonic'], 'Location Name': ['Sand Forest', 'Sand Forest', 'Sand Forest', 'Sand Forest', 'Sand Forest'], 'Coordinates': ['18,961373°S  34,342399°E', '18,961373°S  34,342399°E', '18,961

Chunk-level Operations

In [49]:
#Insert EGCI
from acoustic_knowledge_discovery import features

egci = features.EGCI()
chunkDS= egci(chunkDS, extractor.chunk_size, num_proc=1)


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [50]:
#verify EGCI column properly added
# for row in chunkDS.chunk_ds["train"]:
#     print(row["chunk_id"], row["EGCI"])
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', 'Date', 'Time', 'Duration', 'Season', 'Time of Day', 'Audio Hardware', 'Sound Type', 'Location Name', 'Coordinates', 'Audio Channels', 'Sample Rate', 'Bit Depth', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI'],
        num_rows: 8
    })
})

Binning!

In [51]:
from acoustic_knowledge_discovery.postprocessing import MakeBins2dFloat
makebins = MakeBins2dFloat(chunkDS)
egci_bin={
    "low": ( (0.0, 34), (0.0,0.34) ), 
    "medium": ((0.34,0.67),(0.34,0.67)), 
    "high": ((0.67,1),(0.67,1))
}

chunkDS = makebins("EGCI", egci_bin)

Casting the dataset:   0%|          | 0/8 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [67]:
for row in chunkDS.chunk_ds["train"]:
    #label_str = chunkDS.chunk_ds["train"].features["EGCI"].int2str(row["EGCI"])
    print(row["chunk_id"], row["EGCI"], row["Annotation"])
chunkDS.chunk_ds

SERRA_20250503_152505.wav_0 medium []
SERRA_20250503_160206.wav_0 medium []
SERRA_20250503_160206.wav_5 medium []
SERRA_20250503_220842.wav_0 low ['Cisticola galactotes_201332', 'Cisticola galactotes_201332']
SERRA_20250503_220842.wav_5 low ['Cisticola galactotes_201332', 'Cisticola galactotes_201332']
SERRA_20250503_153828.wav_0 medium []
SERRA_20250503_175145.wav_0 medium []
SERRA_20250503_174918.wav_0 medium ['Mystery mystery_757704']


DatasetDict({
    train: Dataset({
        features: ['file_name', 'Date', 'Time', 'Duration', 'Season', 'Time of Day', 'Audio Hardware', 'Sound Type', 'Location Name', 'Coordinates', 'Audio Channels', 'Sample Rate', 'Bit Depth', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings', 'sample_models/dim_10_muha.pt_embeddings_clusters'],
        num_rows: 8
    })
})

In [53]:
chunkDS.chunk_ds["train"][0]

{'file_name': 'SERRA_20250503_152505.wav',
 'Date': '2025-05-03',
 'Time': '15:25:05',
 'Duration': '00:00:05',
 'Season': 'Transition Rainy to Dry',
 'Time of Day': 'Afternoon',
 'Audio Hardware': 'Wildlife Accoustics',
 'Sound Type': 'Ultrasonic',
 'Location Name': 'Sand Forest',
 'Coordinates': '18,961373°S  34,342399°E',
 'Audio Channels': 'Mono',
 'Sample Rate': '256.0 kHz',
 'Bit Depth': '16-bit',
 'SR': 256000.0,
 'file_path': '/home/s.kamboj.400/mount/files/Field trip 1/Location A (Sand Forrest)/LA_Sand_Forrest_ULTRASONIC_03-05-25/SERRA_20250503_152505.wav',
 'chunk_start': 0,
 'chunk_id': 'SERRA_20250503_152505.wav_0',
 'Annotation': [],
 'Confidence': [],
 'EGCI': 'medium'}

In [ ]:
# # Tests that MakeBins1dFloat does work, but it does not make sense to use it on this dataset
# from acoustic_knowledge_discovery.postprocessing import MakeBins1dFloat
# makebins1d = MakeBins1dFloat(chunkDS)
# OneDBin={
#     "low": (0, 5) , 
#     "medium": (5, 10), 
#     "high": (10, 15)
# }

# chunkDS = makebins1d("chunk_start", OneDBin)

# for row in chunkDS.chunk_ds["train"]:
#     print(row["chunk_id"], row["chunk_start"])
# chunkDS.chunk_ds

# AutoEncoder and Clustering

In [54]:
from acoustic_knowledge_discovery.features import AutoEncoderProcessor

autoEncoder = AutoEncoderProcessor(model_path = "sample_models/dim_10_muha.pt", num_dims=10)
chunkDS = autoEncoder(chunkDS, extractor.chunk_size)
chunkDS.chunk_ds

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file_name', 'Date', 'Time', 'Duration', 'Season', 'Time of Day', 'Audio Hardware', 'Sound Type', 'Location Name', 'Coordinates', 'Audio Channels', 'Sample Rate', 'Bit Depth', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings'],
        num_rows: 8
    })
})

In [55]:
from acoustic_knowledge_discovery.postprocessing import Cluster_API

#HEY HEY HEY: You should note that more num_clusters means more possible nodes
# num_clusters should be some number that is less than your dataset
clustering_pipe = Cluster_API(chunkDS, num_clusters=2)
chunkDS = clustering_pipe("sample_models/dim_10_muha.pt_embeddings")
chunkDS.chunk_ds

DatasetDict({
    train: Dataset({
        features: ['file_name', 'Date', 'Time', 'Duration', 'Season', 'Time of Day', 'Audio Hardware', 'Sound Type', 'Location Name', 'Coordinates', 'Audio Channels', 'Sample Rate', 'Bit Depth', 'SR', 'file_path', 'chunk_start', 'chunk_id', 'Annotation', 'Confidence', 'EGCI', 'sample_models/dim_10_muha.pt_embeddings', 'sample_models/dim_10_muha.pt_embeddings_clusters'],
        num_rows: 8
    })
})

# Convert System to a graph representation

In [56]:
def normalize(v):
    if isinstance(v, list):
        return v
    if isinstance(v, np.ndarray):   # convert numpy array to list
        return v.tolist()
    return [v]  # if it is scalar, then make it list

In [72]:
#test = chunkDS.chunk_ds.select_columns(["file_name", " time_of_day", "season", "species", "chunk_id", "Annotation", "EGCI", "sample_models/dim_10_muha.pt_embeddings_clusters"])
#TODO : change the column names to match dataset
test = chunkDS.chunk_ds.select_columns(["file_name", "Season","Time of Day", "Annotation", "chunk_id", "EGCI", "sample_models/dim_10_muha.pt_embeddings_clusters"])
nodes = test["train"].to_pandas().melt()
nodes = nodes.assign(value=nodes["value"].apply(normalize)).explode("value", ignore_index=True) #necessary so list values are separated
nodes["id"] = nodes["variable"] + "_" + nodes["value"].apply(str)
nodes["group"] = nodes["variable"].apply(lambda x: list(nodes["variable"].unique()).index(x))
nodes[["id", "group"]].head()

,id,group
0,file_name_SERRA_20250503_152505.wav,0
1,file_name_SERRA_20250503_160206.wav,0
2,file_name_SERRA_20250503_160206.wav,0
3,file_name_SERRA_20250503_220842.wav,0
4,file_name_SERRA_20250503_220842.wav,0


In [71]:
edge_list = test["train"].to_pandas().melt(id_vars="file_name")
edge_list = edge_list.assign(value=edge_list["value"].apply(normalize)).explode("value", ignore_index=True)
edge_list = edge_list.dropna(subset=["value"])# drop Nan
edge_list["target"] = edge_list["variable"] + "_" + edge_list["value"].apply(str)
edge_list["source"] = "file_name" + "_" + edge_list["file_name"]
edge_list["value"] = 1
edge_list[["source", "target", "value"]].head()

,source,target,value
0,file_name_SERRA_20250503_152505.wav,Season_Transition Rainy to Dry,1
1,file_name_SERRA_20250503_160206.wav,Season_Transition Rainy to Dry,1
2,file_name_SERRA_20250503_160206.wav,Season_Transition Rainy to Dry,1
3,file_name_SERRA_20250503_220842.wav,Season_Transition Rainy to Dry,1
4,file_name_SERRA_20250503_220842.wav,Season_Transition Rainy to Dry,1


In [70]:
import json

with open("acoustic_knowledge_discovery/d3-visualization/graph_representation.json", mode="w") as f:
    json.dump({
        "nodes": list(nodes[["id", "group"]].drop_duplicates().T.to_dict().values()),  
        "links":  list(edge_list[["source", "target", "value"]].T.to_dict().values()),
    }, f
)

# Visualization

In [62]:
!npm install

m#########⠂⠂⠂⠂⠂⠂⠂⠂⠂) ⠼ idealTree: timing idealTree Completed in 37ms
up to date, audited 13 packages in 333ms

found 0 vulnerabilities


1. In terminal, type ` cd acoustic_knowledge_discovery/d3-visualization/ `, assuming that you are in the acoustic_knowledge_discovery parent directory
2. Run ` npx http-server -p 8080 ` in your terminal*
3. Open ` http://127.0.0.1:8080 ` in Safari/Chrome/your choice of browser to interact with the visualization.


*NOTE: If you are getting the error that npx does not exist, then download it here: https://nodejs.org/en/download

The following 3 lines of code open up the visualization in the jupyter notebook, but it currently only works for MacOS users. 

In [74]:
# command = ["npx", "http-server", "-p", "8081"] 
# target_directory = "acoustic_knowledge_discovery/d3-visualization/"

# process = subprocess.Popen(
#     command,
#     cwd=target_directory,
#     #shell=True,
#     preexec_fn=os.setsid  
# )

# # Persist the group leader PID so you can kill it later even after kernel restarts & kill the entire tree
# with open("/tmp/http_server_pid.json", "w") as f:
#     json.dump({"pgid": process.pid}, f)

# print(f"Server started. PGID={process.pid}")

In [75]:
# #NOTE : You can also open up http://127.0.0.1:8080 in your browser to see it displayed on a larger screen
# HTML(f"""
# <iframe style="background-color: white;" 
#         src="http://127.0.0.1:8080/?nocache={uuid.uuid4()}" 
#         width="1200" height="1000"></iframe>
# """)

In [76]:
# # Kill the entire process group (gracefully) to stop d3vis port
# with open("/tmp/http_server_pid.json") as f:
#     pgid = json.load(f)["pgid"]


# os.killpg(pgid, signal.SIGTERM)

In [77]:
#process.kill()